# Workflow Interface 102 - Held out aggregator validation
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/intel/openfl/blob/develop/openfl-tutorials/experimental/Workflow_Interface_102_Aggregator_Validation.ipynb)

In this tutorial, we build on the ideas from the [first](https://github.com/intel/openfl/blob/develop/openfl-tutorials/experimental/Workflow_Interface_101_MNIST.ipynb) quick start notebook, and demonstrate how to perform validation on the aggregator after training.

# Getting Started

First we start by installing the necessary dependencies for the workflow interface

In [13]:
!pip install git+https://github.com/intel/openfl.git
!pip install -r requirements_workflow_interface.txt
!pip install numpy==1.21.6
!pip install matplotlib
!pip install torchvision
!pip install git+https://github.com/pyviz-topics/imagen.git@master

# Uncomment this if running in Google Colab
#!pip install -r https://raw.githubusercontent.com/intel/openfl/develop/openfl-tutorials/experimental/requirements_workflow_interface.txt
#import os
#os.environ["USERNAME"] = "colab"

  Resolved https://github.com/intel/openfl.git to commit 660cc0260f9fe11641650460c657fe40a29f7442
  Preparing metadata (setup.py) ... done
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/pyviz-topics/imagen.git (to revision master) to /tmp/pip-req-build-32kuebnz
  Running command git clone --filter=blob:none --quiet https://github.com/pyviz-topics/imagen.git /tmp/pip-req-build-32kuebnz
  Resolved https://github.com/pyviz-topics/imagen.git to commit 649e24236aabf0432928a88cbecddf74e1610142
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done


We begin with the quintessential example of a small pytorch CNN model trained on the MNIST dataset. Let's start define our dataloaders, model, optimizer, and some helper functions like we would for any other deep learning experiment

In [14]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import torchvision
import numpy as np

n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

mnist_train = torchvision.datasets.MNIST('files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

mnist_test = torchvision.datasets.MNIST('files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    
def inference(network,test_loader):
    if torch.cuda.is_available():
       network = network.to('cuda:0')
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
      for data, target in test_loader:
        if torch.cuda.is_available():
          data = data.to('cuda:0')
          target = target.to('cuda:0')
        output = network(data)
        test_loss += F.nll_loss(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))
    accuracy = float(correct / len(test_loader.dataset))
    return accuracy

Next we import the `FLSpec`, `LocalRuntime`, and placement decorators.

- `FLSpec` – Defines the flow specification. User defined flows are subclasses of this.
- `Runtime` – Defines where the flow runs, infrastructure for task transitions (how information gets sent). The `LocalRuntime` runs the flow on a single node.
- `aggregator/collaborator` - placement decorators that define where the task will be assigned

In [15]:
from copy import deepcopy

from openfl.experimental.interface import FLSpec, Aggregator, Collaborator
from openfl.experimental.runtime import LocalRuntime
from openfl.experimental.placement import aggregator, collaborator


def FedAvg(models, weights=None):    
    models = [model.to('cpu') for model in models]
    new_model = models[0]
    state_dicts = [model.state_dict() for model in models]
    state_dict = new_model.state_dict()
    for key in models[1].state_dict():
        state_dict[key] = np.average([state[key] for state in state_dicts],axis=0,weights=weights)
    new_model.load_state_dict(state_dict)
    return new_model


Now we come to the updated flow definition. Here we use the same tasks as the [quickstart](https://github.com/psfoley/openfl/blob/experimental-workflow-interface/openfl-tutorials/experimental/Workflow_Interface_MNIST.ipynb), but give the aggregator a `test_loader` as a private attribute. The aggregator will do a forward pass on each of the aggregator's models using it's validation data, and weight the highest accuracy model higher than others.  

Now we come to the updated flow definition. Here we request `@collaborator(num_gpus=1)` as the placement decorator, which will require a dedicated GPU for each collaborator task. Tune this based on your use case, but because this uses Ray internally, you can also pass through a [fraction of a GPU](https://docs.ray.io/en/latest/ray-core/tasks/using-ray-with-gpus.html#fractional-gpus), which will allow more than one task to run on each GPU (i.e. `@collaborator(num_gpus=0.5)` would result in two tasks per GPU). 

In [16]:
class AggregatorValCollaboratorGPUWatermarking(FLSpec):

    def __init__(self, model = None, optimizer = None, rounds=3, **kwargs):
        super().__init__(**kwargs)
        if model is not None:
            self.model = model
            self.optimizer = optimizer
        else:
            self.model = Net()
            self.optimizer = optim.SGD(self.model.parameters(), lr=learning_rate,
                                   momentum=momentum)
        self.rounds = rounds

    @aggregator
    def start(self):
        print(f'Performing initialization for model')
        self.collaborators = self.runtime.collaborators
        self.private = 10
        self.current_round = 0
        self.next(self.aggregated_model_validation,foreach='collaborators',exclude=['private'])

    @collaborator(num_gpus=1)
    def aggregated_model_validation(self):
        print(f'Performing aggregated model validation for collaborator {self.input}')
        self.agg_validation_score = inference(self.model,self.test_loader)
        print(f'{self.input} value of {self.agg_validation_score}')
        self.next(self.train)

    @collaborator(num_gpus=1)
    def train(self):
        self.model.train()
        self.optimizer = optim.SGD(self.model.parameters(), lr=learning_rate,
                                   momentum=momentum)
        train_losses = []
        for batch_idx, (data, target) in enumerate(self.train_loader):
          if torch.cuda.is_available():
              data = data.to("cuda:0")
              target = target.to("cuda:0")
          self.optimizer.zero_grad()
          output = self.model(data)
          loss = F.nll_loss(output, target)
          loss.backward()
          self.optimizer.step()
          if batch_idx % log_interval == 0:
            print('Train Epoch: 1 [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
               batch_idx * len(data), len(self.train_loader.dataset),
              100. * batch_idx / len(self.train_loader), loss.item()))
            self.loss = loss.item()
            torch.save(self.model.state_dict(), 'model.pth')
            torch.save(self.optimizer.state_dict(), 'optimizer.pth')
        self.training_completed = True
        self.next(self.local_model_validation)

    @collaborator(num_gpus=1)
    def local_model_validation(self):
        self.local_validation_score = inference(self.model,self.test_loader)
        print(f'Doing local model validation for collaborator {self.input}: {self.local_validation_score}')
        self.next(self.join, exclude=['training_completed'])

    @aggregator
    def join(self,inputs):
        self.average_loss = sum(input.loss for input in inputs)/len(inputs)
        self.aggregated_model_accuracy = sum(input.agg_validation_score for input in inputs)/len(inputs)
        self.local_model_accuracy = sum(input.local_validation_score for input in inputs)/len(inputs)
        print(f'Average aggregated model validation values = {self.aggregated_model_accuracy}')
        print(f'Average training loss = {self.average_loss}')
        print(f'Average local model validation values = {self.local_model_accuracy}')
        
        highest_accuracy = 0
        highest_accuracy_model_idx = -1
        for idx,col in enumerate(inputs):
            accuracy_for_held_out_agg_data = inference(col.model,self.test_loader)
            if accuracy_for_held_out_agg_data > highest_accuracy:
                highest_accuracy = accuracy_for_held_out_agg_data
                highest_accuracy_model_idx = idx
        
        relative_model_weights = len(inputs)*[1]
        # Give highest accuracy model (on held out aggregator data) 2x the importance
        relative_model_weights[highest_accuracy_model_idx] = 2
        print(f'Aggregator validation score: {highest_accuracy}')
        print(f'Highest accuracy model sent from {inputs[highest_accuracy_model_idx].input}. Receiving 2x weight in updated model')
        self.model = FedAvg([input.model for input in inputs],weights=relative_model_weights)
        self.optimizer = [input.optimizer for input in inputs][0]
        self.current_round += 1
        if self.current_round < self.rounds:
            self.next(self.aggregated_model_validation, foreach='collaborators', exclude=['private'])
        else:
            self.next(self.end)
        
    @aggregator
    def end(self):
        print(f'This is the end of the flow')  

Aggregator step "start" registered
Collaborator step "aggregated_model_validation" registered
Collaborator step "train" registered
Collaborator step "local_model_validation" registered
Aggregator step "join" registered
Aggregator step "end" registered


You'll notice in the `FederatedFlow` definition above that there were certain attributes that the flow was not initialized with, namely the `train_loader` and `test_loader` for each of the collaborators. These are **private_attributes** that are exposed only throught he runtime. Each participant has it's own set of private attributes: a dictionary where the key is the attribute name, and the value is the object that will be made accessible through that participant's task. 

Below, we segment shards of the MNIST dataset for **four collaborators**: Portland, Seattle, Chandler, and Portland. Each has their own slice of the dataset that's accessible via the `train_loader` or `test_loader` attribute. Note that the private attributes are flexible, and you can choose to pass in a completely different type of object to any of the collaborators or aggregator (with an arbitrary name). These private attributes will always be filtered out of the current state when transfering from collaborator to aggregator, or vice versa.  

The LocalRuntime is now initialized **without** the backend argument. Now the LocalRuntime will default to `backend='ray'`, which allows passing through the `num_gpus` argument in the placement decorator

In [17]:
# Setup participants
aggregator = Aggregator()
aggregator.private_attributes = {}

# Setup collaborators with private attributes
collaborator_names = ['Portland', 'Seattle', 'Chandler','Bangalore']
collaborators = [Collaborator(name=name) for name in collaborator_names]

aggregator_test = deepcopy(mnist_test)
aggregator_test.targets = mnist_test.targets[len(collaborators)::len(collaborators)+1]
aggregator_test.data = mnist_test.data[len(collaborators)::len(collaborators)+1]
aggregator.private_attributes = {
    'test_loader': torch.utils.data.DataLoader(aggregator_test,batch_size=batch_size_train, shuffle=True)
}

for idx, collaborator in enumerate(collaborators):
    local_train = deepcopy(mnist_train)
    local_test = deepcopy(mnist_test)
    local_train.data = mnist_train.data[idx::len(collaborators)]
    local_train.targets = mnist_train.targets[idx::len(collaborators)]
    local_test.data = mnist_test.data[idx::len(collaborators)+1]
    local_test.targets = mnist_test.targets[idx::len(collaborators)+1]
    collaborator.private_attributes = {
            'train_loader': torch.utils.data.DataLoader(local_train,batch_size=batch_size_train, shuffle=True),
            'test_loader': torch.utils.data.DataLoader(local_test,batch_size=batch_size_train, shuffle=True)
    }

local_runtime = LocalRuntime(aggregator=aggregator, collaborators=collaborators, backend='single_process')
print(f'Local runtime collaborators = {local_runtime.collaborators}')

Local runtime collaborators = ['Portland', 'Seattle', 'Chandler', 'Bangalore']


Now that we have our flow and runtime defined, let's run the experiment! 

In [18]:
model = None
best_model = None
optimizer = None
flflow = AggregatorValCollaboratorGPUWatermarking(model,optimizer)
flflow.runtime = local_runtime
flflow.run()


Calling start
Performing initialization for model
Sending state from aggregator to collaborators

Calling aggregated_model_validation
Performing aggregated model validation for collaborator Portland


/tmp/ipykernel_6348/615593308.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)



Test set: Avg. loss: 2.3337, Accuracy: 222/2000 (11%)

Portland value of 0.11100000888109207

Calling train
Train Epoch: 1 [0/15000 (0%)]	Loss: 2.369268
Train Epoch: 1 [640/15000 (4%)]	Loss: 2.284822
Train Epoch: 1 [1280/15000 (9%)]	Loss: 2.311420
Train Epoch: 1 [1920/15000 (13%)]	Loss: 2.263354
Train Epoch: 1 [2560/15000 (17%)]	Loss: 2.255242
Train Epoch: 1 [3200/15000 (21%)]	Loss: 2.239205
Train Epoch: 1 [3840/15000 (26%)]	Loss: 2.219298
Train Epoch: 1 [4480/15000 (30%)]	Loss: 2.197631
Train Epoch: 1 [5120/15000 (34%)]	Loss: 2.072708
Train Epoch: 1 [5760/15000 (38%)]	Loss: 2.122330
Train Epoch: 1 [6400/15000 (43%)]	Loss: 2.053948
Train Epoch: 1 [7040/15000 (47%)]	Loss: 1.709623
Train Epoch: 1 [7680/15000 (51%)]	Loss: 1.686352
Train Epoch: 1 [8320/15000 (55%)]	Loss: 1.684603
Train Epoch: 1 [8960/15000 (60%)]	Loss: 1.691072
Train Epoch: 1 [9600/15000 (64%)]	Loss: 1.616916
Train Epoch: 1 [10240/15000 (68%)]	Loss: 1.578318
Train Epoch: 1 [10880/15000 (72%)]	Loss: 1.683727
Train Epoch: 1

/home/albio/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:377: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  a = np.asanyarray(a)
/home/albio/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)



Test set: Avg. loss: 0.6747, Accuracy: 1677/2000 (84%)

Portland value of 0.8385000228881836

Calling train
Train Epoch: 1 [0/15000 (0%)]	Loss: 1.464353
Train Epoch: 1 [640/15000 (4%)]	Loss: 0.923472
Train Epoch: 1 [1280/15000 (9%)]	Loss: 1.025162
Train Epoch: 1 [1920/15000 (13%)]	Loss: 0.736552
Train Epoch: 1 [2560/15000 (17%)]	Loss: 0.878210
Train Epoch: 1 [3200/15000 (21%)]	Loss: 0.956810
Train Epoch: 1 [3840/15000 (26%)]	Loss: 0.958291
Train Epoch: 1 [4480/15000 (30%)]	Loss: 0.987604
Train Epoch: 1 [5120/15000 (34%)]	Loss: 1.022018
Train Epoch: 1 [5760/15000 (38%)]	Loss: 1.024662
Train Epoch: 1 [6400/15000 (43%)]	Loss: 0.895587
Train Epoch: 1 [7040/15000 (47%)]	Loss: 1.169762
Train Epoch: 1 [7680/15000 (51%)]	Loss: 0.759657
Train Epoch: 1 [8320/15000 (55%)]	Loss: 0.759041
Train Epoch: 1 [8960/15000 (60%)]	Loss: 0.719957
Train Epoch: 1 [9600/15000 (64%)]	Loss: 0.953350
Train Epoch: 1 [10240/15000 (68%)]	Loss: 0.803416
Train Epoch: 1 [10880/15000 (72%)]	Loss: 0.551547
Train Epoch: 1

# Congratulations!
Now that you've completed your this notebook, see some of the more advanced things you can do in our [other tutorials](broken_link), including:

- Using the LocalRuntime Ray Backend for dedicated GPU access
- Vertical Federated Learning
- Model Watermarking
- Differential Privacy
- And More!